In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sb
import plotly
import calendar

plotly.offline.init_notebook_mode (connected = True)

In [ ]:
##Import data source ke DataFrame

data = pd.read_csv('/users/macbook/Documents/Kuliah UKM/Open Source/UAS/COVID-19_Case_Surveillance_Public_Use_Data.csv', low_memory=False)

In [ ]:
##Menampilkan atribut kolom data

data.columns

In [ ]:
##Menampilkan row & kolom data

data.shape

In [ ]:
def NaN_info(df):
    global null_view
    try:
        null_view = data[[col for col in data.columns if data[col].isna().sum() > 0]].isna().sum().sort_values(ascending = True)
        null_view = pd.DataFrame(null_view, columns=['NANs'])
        null_view[['PERCENT']] = null_view.NANs.apply(lambda x: round((x/len(df))*100, 2))
        null_view[['TYPE']] = data.dtypes
    except:
        return null_view
    return null_view

NaN_info(data)

In [ ]:
##Menghapus row & kolom yang kosong

data_feature = data.dropna()
data_feature.shape

In [ ]:
##Menampilkan isi kelompok paling atas dari DataFrame

data_feature.head()

In [ ]:
data_feature.describe().transpose()

In [ ]:
##Menampilkan data kelompok berdasarkan kasus lab-confirmed & kasus probable


plt.figure(figsize=(10,8))
sb.countplot(y="age_group",hue='current_status', data=data_feature)

In [ ]:
##Menampilkan nilai tertinggi dan terendah dari kolom cdc_report_dt dalam bentuk date

print("Start date : ", data_feature["cdc_report_dt"].min())
print("Finish date: ", data_feature["cdc_report_dt"].max())

In [ ]:
##Menghitung dan menampilkan jumlah masing2 kolom 

for col in ['current_status', 'sex', 'age_group', 'Race and ethnicity (combined)', 'hosp_yn', 
            'icu_yn', 'death_yn', 'medcond_yn']:
    print("______________________\n")
    print(col)
    print(data_feature[col].value_counts())


In [ ]:
##Mengetahui kelompok umur 

data_feature['age_group'].unique()

In [ ]:
##Menampilkan grafik statistik pasien Covid-19 meninggal dan sembuh menggunakan pie chart dan tabel 

age = ['0 - 9 Years', '10 - 19 Years', '20 - 29 Years', '30 - 39 Years', '40 - 49 Years', 
       '50 - 59 Years', '60 - 69 Years', '70 - 79 Years', '80+ Years']

def create_list(value):
    range_age = []
    for old in value:
        sam = []
        range_old = old
        len_old_yes = len(data_feature[(data_feature['death_yn'] == 'Yes') & (data_feature['age_group'] == old)])
        len_old_no = len(data_feature[(data_feature['death_yn'] == 'No') & (data_feature['age_group'] == old)])
        sam.append(range_old)
        sam.append(len_old_yes)
        sam.append(len_old_no)
        range_age.append(sam)
    return range_age

## Convert list ke DataFrame
new_df = pd.DataFrame(create_list(age), columns=['Kelompok Umur', 'Kematian', 'Kesembuhan'])

## Plotting pie chart Kematian
plt.figure(figsize=(15, 15))
plt.subplot(121)
explode = (0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1) 
plt.pie(new_df['Kematian'], startangle = 90, explode=explode, autopct='%1.0f%%', shadow=True)
plt.legend(labels=new_df['Kelompok Umur'])
plt.title("Kematian pasien Covid-19 antara 2020/01/01 ~ 2020/11/19")

## Plotting pie chart Kesembuhan
plt.subplot(122)
plt.pie(new_df['Kesembuhan'], startangle = 90, explode=explode, autopct='%1.0f%%', shadow=True)
plt.legend(labels=new_df['Kelompok Umur'])
plt.title("Kesembuhan pasien Covid-19 antara 2020/01/01 ~ 2020/11/19")
plt.show()

## Show Table
fig, ax = plt.subplots()
ax.table(cellText=new_df.values, colLabels=new_df.columns, loc="center")
ax.axis('off')
fig.tight_layout()
plt.show()


In [ ]:
##Menampilkan grafik statistik pasien Covid-19 menggunakan 2D Density Plot yang menggambarkan 
##kelompok umur, bulan dan jumlah pasien

data['pos_spec_dt']=pd.to_datetime(data['pos_spec_dt'])
data['Month']=pd.DatetimeIndex(data['pos_spec_dt']).month
df_plot = data.sample(frac=0.4).copy()
fig2 = px.density_contour(df_plot, x="Month", y="age_group")
fig2.update_traces(contours_coloring="fill", contours_showlabels = True)
fig2.show()

In [ ]:
##Menampilkan grafik statistik pasien Covid-19 menggunakan Parallel Coordinate Plot

fig3 = px.parallel_categories(df_plot, dimensions=['sex','age_group','death_yn'],
                 color_continuous_scale=px.colors.sequential.Inferno,
                labels={'sex':'Jenis Kelamin', 'age_group':'Kelompok Umur', 'death_yn':'Status Kematian'})
fig3.show()